In [29]:
import os

In [30]:
print(os.getcwd())

/Users/williamvanbuskirk/Documents/2 - Library/library_query/tulip-library-search/notebooks


In [34]:
import redshift_connector
import pandas as pd
import os
import requests
import json
import numpy as np
HOST = os.getenv('REDSHIFT_HOST')
DATABASE = os.getenv('REDSHIFT_DB')
USER = os.getenv('REDSHIFT_USER')
PASSWORD = os.getenv('REDSHIFT_PASSWORD')

conn = redshift_connector.connect(
    host=HOST,
    database=DATABASE,
    user=USER,
    password=PASSWORD
)

cursor = conn.cursor()

query = """
SELECT asset_id, asset_page_url  FROM cds.tbllibraryasset
"""
cursor.execute(query)
library_assets = cursor.fetch_dataframe()
cursor.close()
conn.close()

# recently added content
response = requests.get('https://api.library.tulipintra.net/v2/assets')
body = response.content
body_json = json.loads(body)  
apps = body_json['apps']
df_apps = pd.DataFrame(apps)

app_groups = body_json['appGroups']
df_app_groups = pd.DataFrame(app_groups)

connectors = body_json['connectors']
df_connectors = pd.DataFrame(connectors)

df_union = pd.concat([df_apps, df_app_groups, df_connectors])
#df_union.columns
df_union_col = ['assetId', 'assetType', 'created', 'lastModified', 'marketingHeadline', 'name']
df_union = df_union[df_union_col] 

recent_asset_table = pd.merge(library_assets, df_union, left_on='asset_id', right_on='assetId')
recent_asset_table.columns

Index(['asset_id', 'asset_page_url', 'assetId', 'assetType', 'created',
       'lastModified', 'marketingHeadline', 'name'],
      dtype='object')

In [44]:

recent_asset_table = recent_asset_table.drop(columns=['asset_id', 'assetId'])

recent_asset_table['date addded, updated'] = np.where(recent_asset_table['lastModified'].isnull(), recent_asset_table['created'], recent_asset_table['lastModified'])



recent_asset_table = recent_asset_table[['name','assetType', 'marketingHeadline', 'date addded, updated', 'asset_page_url']]



KeyError: 'lastModified'

In [45]:
recent_asset_table['date addded, updated'] = recent_asset_table['date addded, updated'].apply(lambda x: x['at'] if 'at' in x else None)

In [37]:
recent_asset_table = recent_asset_table.rename(columns={
    'assetType': 'content type',
    'marketingHeadline': 'summary',
    'asset_page_url':'link'
})



In [42]:
recent_asset_table['date addded, updated']['at']

KeyError: 'at'

In [46]:
recent_asset_table = recent_asset_table.sort_values(by='date addded, updated', ascending=False)

recent_asset_table = recent_asset_table.reset_index(drop=True)


# Modify the index to start with 1 instead of 0
recent_asset_table.index = recent_asset_table.index + 1